In [1]:
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
from pathlib import Path
import os
import cv2
from plyfile import PlyData
from tqdm import tqdm

from scipy.spatial.transform import Rotation as R, Slerp
from concurrent.futures import ThreadPoolExecutor

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def read_ply(file_path):
    plydata = PlyData.read(file_path)
    x = plydata['vertex']['x']
    y = plydata['vertex']['y']
    z = plydata['vertex']['z']
    if 'red' in plydata['vertex']:
        r = plydata['vertex']['red']
        g = plydata['vertex']['green']
        b = plydata['vertex']['blue']
        colors = np.array([r, g, b]).T
    else:
        colors = None
    if 'nx' in plydata['vertex']:
        nx = plydata['vertex']['nx']
        ny = plydata['vertex']['ny']
        nz = plydata['vertex']['nz']
        normals = np.array([nx, ny, nz]).T
    else:
        normals = None
    points = np.array([x, y, z]).T
    print(f"Number of points: {points.shape[0]}")
    return points, colors, normals

In [3]:
#
# Copyright (C) 2023, Inria
# GRAPHDECO research group, https://team.inria.fr/graphdeco
# All rights reserved.
#
# This software is free for non-commercial, research and evaluation use 
# under the terms of the LICENSE.md file.
#
# For inquiries contact  george.drettakis@inria.fr
#

import numpy as np
import collections
import struct

CameraModel = collections.namedtuple(
    "CameraModel", ["model_id", "model_name", "num_params"])
Camera = collections.namedtuple(
    "Camera", ["id", "model", "width", "height", "params"])
BaseImage = collections.namedtuple(
    "Image", ["id", "qvec", "tvec", "camera_id", "name", "xys", "point3D_ids"])
Point3D = collections.namedtuple(
    "Point3D", ["id", "xyz", "rgb", "error", "image_ids", "point2D_idxs"])
CAMERA_MODELS = {
    CameraModel(model_id=0, model_name="SIMPLE_PINHOLE", num_params=3),
    CameraModel(model_id=1, model_name="PINHOLE", num_params=4),
    CameraModel(model_id=2, model_name="SIMPLE_RADIAL", num_params=4),
    CameraModel(model_id=3, model_name="RADIAL", num_params=5),
    CameraModel(model_id=4, model_name="OPENCV", num_params=8),
    CameraModel(model_id=5, model_name="OPENCV_FISHEYE", num_params=8),
    CameraModel(model_id=6, model_name="FULL_OPENCV", num_params=12),
    CameraModel(model_id=7, model_name="FOV", num_params=5),
    CameraModel(model_id=8, model_name="SIMPLE_RADIAL_FISHEYE", num_params=4),
    CameraModel(model_id=9, model_name="RADIAL_FISHEYE", num_params=5),
    CameraModel(model_id=10, model_name="THIN_PRISM_FISHEYE", num_params=12)
}
CAMERA_MODEL_IDS = dict([(camera_model.model_id, camera_model)
                         for camera_model in CAMERA_MODELS])
CAMERA_MODEL_NAMES = dict([(camera_model.model_name, camera_model)
                           for camera_model in CAMERA_MODELS])


def qvec2rotmat(qvec):
    return np.array([
        [1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
         2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
         2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]],
        [2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
         1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
         2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]],
        [2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
         2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
         1 - 2 * qvec[1]**2 - 2 * qvec[2]**2]])

def rotmat2qvec(R):
    Rxx, Ryx, Rzx, Rxy, Ryy, Rzy, Rxz, Ryz, Rzz = R.flat
    K = np.array([
        [Rxx - Ryy - Rzz, 0, 0, 0],
        [Ryx + Rxy, Ryy - Rxx - Rzz, 0, 0],
        [Rzx + Rxz, Rzy + Ryz, Rzz - Rxx - Ryy, 0],
        [Ryz - Rzy, Rzx - Rxz, Rxy - Ryx, Rxx + Ryy + Rzz]]) / 3.0
    eigvals, eigvecs = np.linalg.eigh(K)
    qvec = eigvecs[[3, 0, 1, 2], np.argmax(eigvals)]
    if qvec[0] < 0:
        qvec *= -1
    return qvec

class Image(BaseImage):
    def qvec2rotmat(self):
        return qvec2rotmat(self.qvec)

def read_next_bytes(fid, num_bytes, format_char_sequence, endian_character="<"):
    """Read and unpack the next bytes from a binary file.
    :param fid:
    :param num_bytes: Sum of combination of {2, 4, 8}, e.g. 2, 6, 16, 30, etc.
    :param format_char_sequence: List of {c, e, f, d, h, H, i, I, l, L, q, Q}.
    :param endian_character: Any of {@, =, <, >, !}
    :return: Tuple of read and unpacked values.
    """
    data = fid.read(num_bytes)
    return struct.unpack(endian_character + format_char_sequence, data)

def read_points3D_text(path):
    """
    see: src/base/reconstruction.cc
        void Reconstruction::ReadPoints3DText(const std::string& path)
        void Reconstruction::WritePoints3DText(const std::string& path)
    """
    xyzs = None
    rgbs = None
    errors = None
    num_points = 0
    with open(path, "r") as fid:
        while True:
            line = fid.readline()
            if not line:
                break
            line = line.strip()
            if len(line) > 0 and line[0] != "#":
                num_points += 1


    xyzs = np.empty((num_points, 3))
    rgbs = np.empty((num_points, 3))
    errors = np.empty((num_points, 1))
    count = 0
    with open(path, "r") as fid:
        while True:
            line = fid.readline()
            if not line:
                break
            line = line.strip()
            if len(line) > 0 and line[0] != "#":
                elems = line.split()
                xyz = np.array(tuple(map(float, elems[1:4])))
                rgb = np.array(tuple(map(int, elems[4:7])))
                error = np.array(float(elems[7]))
                xyzs[count] = xyz
                rgbs[count] = rgb
                errors[count] = error
                count += 1

    return xyzs, rgbs, errors

def read_points3D_binary(path_to_model_file):
    """
    see: src/base/reconstruction.cc
        void Reconstruction::ReadPoints3DBinary(const std::string& path)
        void Reconstruction::WritePoints3DBinary(const std::string& path)
    """


    with open(path_to_model_file, "rb") as fid:
        num_points = read_next_bytes(fid, 8, "Q")[0]

        xyzs = np.empty((num_points, 3))
        rgbs = np.empty((num_points, 3))
        errors = np.empty((num_points, 1))

        for p_id in range(num_points):
            binary_point_line_properties = read_next_bytes(
                fid, num_bytes=43, format_char_sequence="QdddBBBd")
            xyz = np.array(binary_point_line_properties[1:4])
            rgb = np.array(binary_point_line_properties[4:7])
            error = np.array(binary_point_line_properties[7])
            track_length = read_next_bytes(
                fid, num_bytes=8, format_char_sequence="Q")[0]
            track_elems = read_next_bytes(
                fid, num_bytes=8*track_length,
                format_char_sequence="ii"*track_length)
            xyzs[p_id] = xyz
            rgbs[p_id] = rgb
            errors[p_id] = error
    return xyzs, rgbs, errors

def read_intrinsics_text(path):
    """
    Taken from https://github.com/colmap/colmap/blob/dev/scripts/python/read_write_model.py
    """
    cameras = {}
    with open(path, "r") as fid:
        while True:
            line = fid.readline()
            if not line:
                break
            line = line.strip()
            if len(line) > 0 and line[0] != "#":
                elems = line.split()
                camera_id = int(elems[0])
                model = elems[1]
                assert model == "PINHOLE", "While the loader support other types, the rest of the code assumes PINHOLE"
                width = int(elems[2])
                height = int(elems[3])
                params = np.array(tuple(map(float, elems[4:])))
                cameras[camera_id] = Camera(id=camera_id, model=model,
                                            width=width, height=height,
                                            params=params)
    return cameras

def read_extrinsics_binary(path_to_model_file):
    """
    see: src/base/reconstruction.cc
        void Reconstruction::ReadImagesBinary(const std::string& path)
        void Reconstruction::WriteImagesBinary(const std::string& path)
    """
    images = {}
    with open(path_to_model_file, "rb") as fid:
        num_reg_images = read_next_bytes(fid, 8, "Q")[0]
        for _ in range(num_reg_images):
            binary_image_properties = read_next_bytes(
                fid, num_bytes=64, format_char_sequence="idddddddi")
            image_id = binary_image_properties[0]
            qvec = np.array(binary_image_properties[1:5])
            tvec = np.array(binary_image_properties[5:8])
            camera_id = binary_image_properties[8]
            image_name = ""
            current_char = read_next_bytes(fid, 1, "c")[0]
            while current_char != b"\x00":   # look for the ASCII 0 entry
                image_name += current_char.decode("utf-8")
                current_char = read_next_bytes(fid, 1, "c")[0]
            num_points2D = read_next_bytes(fid, num_bytes=8,
                                           format_char_sequence="Q")[0]
            x_y_id_s = read_next_bytes(fid, num_bytes=24*num_points2D,
                                       format_char_sequence="ddq"*num_points2D)
            xys = np.column_stack([tuple(map(float, x_y_id_s[0::3])),
                                   tuple(map(float, x_y_id_s[1::3]))])
            point3D_ids = np.array(tuple(map(int, x_y_id_s[2::3])))
            images[image_id] = Image(
                id=image_id, qvec=qvec, tvec=tvec,
                camera_id=camera_id, name=image_name,
                xys=xys, point3D_ids=point3D_ids)
    return images


def read_intrinsics_binary(path_to_model_file):
    """
    see: src/base/reconstruction.cc
        void Reconstruction::WriteCamerasBinary(const std::string& path)
        void Reconstruction::ReadCamerasBinary(const std::string& path)
    """
    cameras = {}
    with open(path_to_model_file, "rb") as fid:
        num_cameras = read_next_bytes(fid, 8, "Q")[0]
        for _ in range(num_cameras):
            camera_properties = read_next_bytes(
                fid, num_bytes=24, format_char_sequence="iiQQ")
            camera_id = camera_properties[0]
            model_id = camera_properties[1]
            model_name = CAMERA_MODEL_IDS[camera_properties[1]].model_name
            width = camera_properties[2]
            height = camera_properties[3]
            num_params = CAMERA_MODEL_IDS[model_id].num_params
            params = read_next_bytes(fid, num_bytes=8*num_params,
                                     format_char_sequence="d"*num_params)
            cameras[camera_id] = Camera(id=camera_id,
                                        model=model_name,
                                        width=width,
                                        height=height,
                                        params=np.array(params))
        assert len(cameras) == num_cameras
    return cameras


def read_extrinsics_text(path):
    """
    Taken from https://github.com/colmap/colmap/blob/dev/scripts/python/read_write_model.py
    """
    images = {}
    with open(path, "r") as fid:
        while True:
            line = fid.readline()
            if not line:
                break
            line = line.strip()
            if len(line) > 0 and line[0] != "#":
                elems = line.split()
                image_id = int(elems[0])
                qvec = np.array(tuple(map(float, elems[1:5])))
                tvec = np.array(tuple(map(float, elems[5:8])))
                camera_id = int(elems[8])
                image_name = elems[9]
                elems = fid.readline().split()
                xys = np.column_stack([tuple(map(float, elems[0::3])),
                                       tuple(map(float, elems[1::3]))])
                point3D_ids = np.array(tuple(map(int, elems[2::3])))
                images[image_id] = Image(
                    id=image_id, qvec=qvec, tvec=tvec,
                    camera_id=camera_id, name=image_name,
                    xys=xys, point3D_ids=point3D_ids)
    return images


def read_colmap_bin_array(path):
    """
    Taken from https://github.com/colmap/colmap/blob/dev/scripts/python/read_dense.py

    :param path: path to the colmap binary file.
    :return: nd array with the floating point values in the value
    """
    with open(path, "rb") as fid:
        width, height, channels = np.genfromtxt(fid, delimiter="&", max_rows=1,
                                                usecols=(0, 1, 2), dtype=int)
        fid.seek(0)
        num_delimiter = 0
        byte = fid.read(1)
        while True:
            if byte == b"&":
                num_delimiter += 1
                if num_delimiter >= 3:
                    break
            byte = fid.read(1)
        array = np.fromfile(fid, np.float32)
    array = array.reshape((width, height, channels), order="F")
    return np.transpose(array, (1, 0, 2)).squeeze()

In [4]:
def project_points(points_3d, camera_intrinsics, extrinsics):
    """
    Project 3D points onto a 2D image plane using camera intrinsics and extrinsics.
    
    Parameters:
    - points_3d: np.ndarray - Array of 3D points (N, 3).
    - camera_intrinsics: Camera - The camera intrinsics (e.g., focal length, principal point).
    - extrinsics: np.ndarray - 4x4 extrinsic matrix (rotation and translation).

    Returns:
    - projected_points: np.ndarray - 2D projected points in image space.
    - mask: np.ndarray - Boolean array indicating which points are in front of the camera.
    """
    # Convert points to homogeneous coordinates
    points_homogeneous = np.hstack((points_3d, np.ones((points_3d.shape[0], 1))))
    
    # Transform 3D points to the camera coordinate system
    points_camera = (extrinsics @ points_homogeneous.T).T
    points_camera = points_camera[:, :3]

    # Filter points in front of the camera
    mask = points_camera[:, 2] > 0  # Keep points with positive Z
    points_camera = points_camera[mask]

    # Create intrinsic matrix
    fx, fy, cx, cy = camera_intrinsics.params  # Assuming Pinhole model (fx, fy, cx, cy)
    K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

    # Project points onto the image plane
    projected_points = (K @ points_camera.T).T
    projected_points = projected_points[:, :2] / projected_points[:, 2].reshape(-1, 1)


    # Return only the points within the image boundaries
    return projected_points.astype(int), mask

def get_extrinsic_matrix(qvec, tvec):
    """
    Create a 4x4 extrinsic matrix from quaternion and translation vector.
    
    Parameters:
    - qvec: np.ndarray - Quaternion (w, x, y, z) representing rotation.
    - tvec: np.ndarray - Translation vector (x, y, z).
    
    Returns:
    - extrinsic_matrix: np.ndarray - 4x4 extrinsic matrix.
    """
    # Convert quaternion to rotation matrix
    rotation_matrix = qvec2rotmat(qvec)
    
    # Create 4x4 extrinsic matrix
    extrinsic_matrix = np.eye(4)
    extrinsic_matrix[:3, :3] = rotation_matrix
    extrinsic_matrix[:3, 3] = tvec
    
    return extrinsic_matrix

In [21]:
def project_point_cloud(point_cloud, colors, camera_intrinsics, extrinsics):
    """
    Project a 3D point cloud onto a 2D image plane.

    Args:
        point_cloud (np.ndarray): Nx3 array of 3D points.
        camera_intrinsics (Camera): Camera object with intrinsic parameters.
        extrinsics (np.ndarray): 4x4 matrix for camera extrinsics.

    Returns:
        projected_points (np.ndarray): Mx2 array of 2D projected points (filtered by visibility).
        valid_mask (np.ndarray): Boolean mask indicating the visible points (length matches filtered points).
    """
    if extrinsics.shape != (4, 4):
        raise ValueError("Extrinsics must be a 4x4 matrix.")

    # Add a column of ones to the point cloud to make it homogeneous
    points_homogeneous = np.hstack((point_cloud, np.ones((point_cloud.shape[0], 1))))

    # Transform points to the camera coordinate system using extrinsics
    points_camera = (extrinsics @ points_homogeneous.T).T
    points_camera = points_camera[:, :3]  # Extract x, y, z coordinates

    # Keep only points with positive Z (in front of the camera)
    visibility_mask = points_camera[:, 2] > 0
    points_camera = points_camera[visibility_mask]
    filtered_colors = colors[visibility_mask]

    # Project points onto the 2D image plane using the intrinsic matrix
    fx, fy, cx, cy = camera_intrinsics.params
    K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
    
    projected_points = (K @ points_camera.T).T
    projected_points = projected_points[:, :2] / projected_points[:, 2].reshape(-1, 1)

    # Return filtered projected points and the valid mask
    return projected_points.astype(int), filtered_colors


In [6]:
def render_point_cloud_image(point_cloud, camera, extrinsics, image_path, image_size=None, point_color=(0, 255, 0), alpha=0.5, point_size=2):
    """
    Render a composite image with the projected point cloud and a masked original image.

    Args:
        point_cloud (np.ndarray): Nx3 array of 3D points.
        camera (Camera): Camera object with intrinsics.
        extrinsics (np.ndarray): 4x4 camera extrinsic matrix.
        image_path (str): Path to the input background image.
        output_path (str): Path to save the output composite image.
        image_size (tuple): Size of the output image (height, width).
        point_color (tuple): RGB color of the points to render.
        alpha (float): Opacity of the overlay.
        point_size (int): Size of the projected points.
    """
    if image_size is None:
        image_size = (camera.height, camera.width)
    
    # Project the 3D points onto the 2D image plane
    projected_points = project_point_cloud(point_cloud, camera, extrinsics)

    # Create the point cloud image
    point_cloud_image = np.zeros((image_size[0], image_size[1], 3), dtype=np.uint8)
    
    for point in projected_points:
        cv2.circle(point_cloud_image, tuple(point), point_size, point_color, -1)

    # Load the actual image
    actual_image = cv2.imread(image_path)
    if actual_image is None:
        raise FileNotFoundError(f"Image not found: {image_path}")

    if actual_image.shape[:2] != image_size:
        actual_image = cv2.resize(actual_image, (image_size[1], image_size[0]))

    # Overlay the point cloud on the actual image
    overlay_image = cv2.addWeighted(point_cloud_image, alpha, actual_image, 1 - alpha, 0)
    return overlay_image


In [26]:
def render_point_cloud_interpolated(point_cloud, colors, camera, extrinsics, output_path, image_size=None, alpha=0.5, point_size=1):
    """
    Render a composite image with the projected point cloud and a masked original image.

    Args:
        point_cloud (np.ndarray): Nx3 array of 3D points.
        camera (Camera): Camera object with intrinsics.
        extrinsics (np.ndarray): 4x4 camera extrinsic matrix.
        image_path (str): Path to the input background image.
        output_path (str): Path to save the output composite image.
        image_size (tuple): Size of the output image (height, width).
        point_color (tuple): RGB color of the points to render.
        alpha (float): Opacity of the overlay.
        point_size (int): Size of the projected points.
    """
    if image_size is None:
        image_size = (camera.height, camera.width)
    
    # Initialize the depth buffer and output image
    depth_buffer = np.full(image_size, np.inf)  # Depth buffer initialized to infinity
    point_cloud_image = np.zeros((image_size[0], image_size[1], 3), dtype=np.uint8)

    # Project the 3D points onto the 2D image plane
    projected_points, colors = project_point_cloud(point_cloud, colors, camera, extrinsics)

    points_camera = (extrinsics @ np.hstack((point_cloud, np.ones((point_cloud.shape[0], 1)))).T).T

    # Iterate through projected points and apply depth testing
    for idx, (u, v) in enumerate(projected_points):
        z = points_camera[idx, 2]  # Depth value
        u, v = int(round(u)), int(round(v))

        # Check if point is within image bounds
        if 0 <= u < image_size[1] and 0 <= v < image_size[0]:
            # Depth test: Render only if this point is closer
            if z < depth_buffer[v, u]:
                depth_buffer[v, u] = z  # Update depth buffer
                color = colors[idx]
                point_color = (int(color[2]), int(color[1]), int(color[0]))
                cv2.circle(point_cloud_image, (u, v), point_size, point_color, -1)

    # Save the final image
    cv2.imwrite(output_path, point_cloud_image)
    return point_cloud_image


In [8]:
def project_points_to_image(camera_id, cam_extrinsics, cam_intrinsics, object_name, points):
    camera = cam_intrinsics[camera_id] if camera_id in cam_intrinsics else cam_intrinsics[list(cam_intrinsics.keys())[0]]
    qvec = cam_extrinsics[camera_id].qvec
    tvec = cam_extrinsics[camera_id].tvec
    extrinsics = get_extrinsic_matrix(qvec, tvec)
    name = cam_extrinsics[camera_id].name
    image = render_point_cloud_image(points, camera, extrinsics, image_path)
    return image

In [11]:


def slerp(q1, q2, t):
    """
    Spherical Linear Interpolation between two quaternions.
    """
    q1 = R.from_quat(q1)
    q2 = R.from_quat(q2)
    slerp_instance = Slerp([0, 1], R.from_quat([q1.as_quat(), q2.as_quat()]))
    return slerp_instance([t])[0].as_quat()

def interpolate_camera_poses(camera_ids, cam_extrinsics, num_samples=200, zoom_factor=1.0):
    """
    Interpolates camera poses with an option to adjust the path for a zoomed-out effect.

    Parameters:
    - camera_ids: List of camera IDs for interpolation.
    - cam_extrinsics: Dictionary containing camera extrinsics (qvec and tvec).
    - num_samples: Number of interpolated poses to generate.
    - zoom_factor: Multiplier for "zooming out" the camera path. Values >1 push cameras further out.

    Returns:
    - List of interpolated poses as tuples (qvec, tvec).
    """
    poses = []
    for i in range(len(camera_ids) - 1):
        q1 = cam_extrinsics[camera_ids[i]].qvec
        t1 = cam_extrinsics[camera_ids[i]].tvec
        q2 = cam_extrinsics[camera_ids[i + 1]].qvec
        t2 = cam_extrinsics[camera_ids[i + 1]].tvec

        # Compute the direction vector for zoom adjustment
        direction_vector = t2 - t1

        for j in range(num_samples // (len(camera_ids) - 1)):
            t = j / (num_samples // (len(camera_ids) - 1))
            interpolated_q = slerp(q1, q2, t)
            interpolated_t = (1 - t) * t1 + t * t2

            # Apply zoom factor to the translation vector
            center_t = (t1 + t2) / 2  # Find the midpoint for zoom adjustment
            zoomed_t = interpolated_t + (interpolated_t - center_t) * (zoom_factor - 1)

            poses.append((interpolated_q, zoomed_t))
    return poses


def render_interpolated_video(camera_ids, cam_extrinsics, cam_intrinsics, points, colors, num_samples=200, output_dir="output", zoom_factor=1.0):
    interpolated_poses = interpolate_camera_poses(camera_ids, cam_extrinsics, num_samples, zoom_factor)
    image_size = (cam_intrinsics[1].height, cam_intrinsics[1].width)
    for i, (qvec, tvec) in enumerate(tqdm(interpolated_poses, desc="Rendering Frames", unit="frame")):
        extrinsics = get_extrinsic_matrix(qvec, tvec)
        camera_id = 1  # Use the first camera's intrinsics
        camera = cam_intrinsics[camera_id]
        image_path = f"{output_dir}/frame_{i:04d}.png"
        render_point_cloud_interpolated(points, colors, camera, extrinsics, image_path, image_size=image_size)
    print(f"Rendered {len(interpolated_poses)} frames.")

In [27]:
# Define camera positions and orientations
camera_ids = [1, 280]
camera_positions = np.array([
    [0.747852111589277, -2.1887432598651739, 3.3341802907528604],  # Camera 1 position
    [0.31344161331034526, -2.1952116629036906, 2.8254937930775985],  # Camera 2 position
    [0.17202464003031009, -1.8970149533814831, 2.880851400336415],   # Camera 3 position
    [4.63097, -2.10853, -6.58023]
])

camera_orientations = [
    R.from_quat([-0.13577322541282349, -0.40936959454237748, -0.18740952320445933, 0.88253036034885812]),  # Camera 1
    R.from_quat([-0.11390566885617641, -0.076043765232894661, -0.061089609451369535, 0.98869151103299691]),  # Camera 2
    R.from_quat([-0.13913855551931867, 0.52118708493071131, 0.26265739115675008, 0.80000973727833791]),      # Camera 3
    R.from_quat([-0.339762, 0.580685, -0.736785, 0.067198])
]

object_name = "kitchen"
points, colors, normals = read_ply(f'data/points_{object_name}.ply')
cam_extrinsics = read_extrinsics_text(f"data/{object_name}/sparse/0/images.txt")
cam_intrinsics = read_intrinsics_text(f"data/{object_name}/sparse/0/cameras.txt")
# Sample Usage
render_interpolated_video(camera_ids, cam_extrinsics, cam_intrinsics, points, colors, num_samples=200, output_dir=f"output/{object_name}/videoframes2", zoom_factor=1)


Number of points: 241367


Rendering Frames: 100%|██████████| 200/200 [03:26<00:00,  1.03s/frame]

Rendered 200 frames.
